In [38]:
import numpy as np
import pandas as pd
import librosa
import sklearn
import os
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [39]:
# names = []
# Letter_names = ["M", "S", "B", "Y"]
# for letter in Letter_names:
#     for index in range(15):
#         names.append({'file':"{0}_open_({1}).wav".format(letter, index), 'label':"{0}".format(letter)})
# file_name = pd.DataFrame(names)
# print(file_name)

In [40]:
Names = os.listdir("AllRecords//close")
Close_Names_df = pd.DataFrame(Names, columns={'Paths'})
Close_Names_df.insert(1, "Labels", len(Names) *[0])
print(Close_Names_df)

                                                 Paths  Labels
0    AuphonicRecording_Sherif Close The Window (1).wav       0
1    AuphonicRecording_Sherif Close The Window (2).wav       0
2    AuphonicRecording_Sherif Close The Window (3).wav       0
3    AuphonicRecording_Sherif Close The Window (4).wav       0
4    AuphonicRecording_Sherif Close The Window (5).wav       0
..                                                 ...     ...
259                                    y_close_(5).wav       0
260                                    y_close_(6).wav       0
261                                    y_close_(7).wav       0
262                                    y_close_(8).wav       0
263                                    y_close_(9).wav       0

[264 rows x 2 columns]


In [41]:
#Getting Team information
Names2 = os.listdir("AllRecords//open")
Open_Names_df = pd.DataFrame(Names2, columns={'Paths'})
Open_Names_df.insert(1, "Labels", len(Names2) *[1])
print(Open_Names_df)
all = Open_Names_df.append(Close_Names_df, ignore_index=True)
print(all)

                                               Paths  Labels
0    AuphonicRecording_Sherief Open The Door (1).wav       1
1    AuphonicRecording_Sherief Open The Door (2).wav       1
2    AuphonicRecording_Sherief Open The Door (3).wav       1
3    AuphonicRecording_Sherief Open The Door (4).wav       1
4    AuphonicRecording_Sherief Open The Door (5).wav       1
..                                               ...     ...
279                    yassmen open the door (5).wav       1
280                    yassmen open the door (6).wav       1
281                    yassmen open the door (7).wav       1
282                    yassmen open the door (8).wav       1
283                    yassmen open the door (9).wav       1

[284 rows x 2 columns]
                                               Paths  Labels
0    AuphonicRecording_Sherief Open The Door (1).wav       1
1    AuphonicRecording_Sherief Open The Door (2).wav       1
2    AuphonicRecording_Sherief Open The Door (3).wav       1


C:\Users\mahmo\AppData\Local\Temp\ipykernel_25504\4113759299.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all = Open_Names_df.append(Close_Names_df, ignore_index=True)


In [42]:
def Extract_features(paths, whichone, dowhat = ''):
    path = os.path.join(os.path.abspath('AllRecords/{0}').format(whichone)+ ('\\') +str(paths['Paths']))
    # X, sample_rate = librosa.load(path, res_type = "kaiser_fast")
    # X, index    = librosa.effects.trim(X)
    # chroma_stft = librosa.feature.chroma_stft(y=X, sr=sample_rate)
    # rmse = librosa.feature.rms(y=X)
    # spec_cent = librosa.feature.spectral_centroid(y=X, sr=sample_rate) 
    # spec_bw = librosa.feature.spectral_bandwidth(y=X, sr=sample_rate)
    # rolloff = librosa.feature.spectral_rolloff(y=X, sr=sample_rate)
    # zcr = librosa.feature.zero_crossing_rate(y=X)
    # mfcc = librosa.feature.mfcc(y=X, sr=sample_rate,n_mfcc=40) 
    # to_append   = f'{np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'
    # for e in mfcc:
    #     to_append += f' {np.mean(e)}'
    # features=[]
    # features.append(to_append.split())
    # print(len(features[0]))

    # for counter in range(0,len(features[0])):
    #     features[0][counter]=float(features[0][counter])
    # # return [chroma_stft, rmse, spec_cent, spec_bw, rolloff,zcr,mfcc]
    # # return [chroma_stft, rmse, spec_cent, spec_bw, rolloff,zcr,mfcc]
    # return features
    # features=[]
    # reading records
    y, sr = librosa.load(path,res_type='kaiser_fast')
    # remove leading and trailing silence
    y, index    = librosa.effects.trim(y)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse        = librosa.feature.rms(y=y)
    spec_cent   = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw     = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff     = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr         = librosa.feature.zero_crossing_rate(y)
    mfcc        = librosa.feature.mfcc(y=y, sr=sr)
    # to_append   = f'{np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'
    # for e in mfcc:
    #     to_append += f' {np.mean(e)}'
    
    # features.append(to_append.split())
    # # print(len(features[0]))
    
    # for counter in range(0,len(features[0])):
    #     features[0][counter]=float(features[0][counter])

    # return features
    return [chroma_stft, rmse, spec_cent, spec_bw, rolloff,zcr,mfcc]

In [43]:
mfccs = []
mel = []
tonnetz = []
chroma = []
contrast = []
label_enter = []
DataSet = pd.DataFrame()
DataSet = all.apply(Extract_features, whichone="AllTeams", axis=1)
dd = np.array(DataSet[0])
print(DataSet)

0      [[[0.10397341, 0.1377187, 0.0813543, 0.0807806...
1      [[[0.94728476, 0.6589324, 0.91914415, 1.0, 0.5...
2      [[[0.2787472, 0.5081707, 0.41435555, 0.328101,...
3      [[[1.0, 1.0, 0.99109715, 0.93272287, 0.4674527...
4      [[[0.43665874, 0.4805539, 0.43445337, 0.512618...
                             ...                        
543    [[[1.0, 0.82806844, 0.95903087, 1.0, 0.924533,...
544    [[[0.53762746, 0.7825138, 0.8607997, 1.0, 0.58...
545    [[[0.87527734, 0.83723205, 0.7751678, 0.787379...
546    [[[0.79039603, 0.7560549, 0.66985595, 1.0, 0.9...
547    [[[0.44455808, 0.62409323, 0.6815331, 0.826901...
Length: 548, dtype: object


C:\Users\mahmo\AppData\Local\Temp\ipykernel_25504\3386549790.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dd = np.array(DataSet[0])


In [44]:
def features_append(features):
    feature = []
    # features[0][5] = [features[0][5]]
    for i in range(len(features)):
        # print(features[0][1],'1')
        # print(features[0][2], '2')
        # print(features[0][3], '3')
        # print(features[0][4], '4')
        # print(features[0][5], '5')
        # feature.append(np.concatenate((features[i][0], features[i][1], features[i][2]
        # , features[i][3],features[i][4],features[i][5], features[i][6]), axis=0))
        to_append   = f'{np.mean(features[i][0])} {np.mean(features[i][1])} {np.mean(features[i][2])} {np.mean(features[i][3])} {np.mean(features[i][4])} {np.mean(features[i][5])}'
        for e in features[i][6]:
            to_append += f' {np.mean(e)}'
        # print(to_append.split())
        feature.append(to_append.split())
        if(i==0):
            print(feature)

    # print("These are: ",features)
    # print("The length",len(features[0]))
    # print(feature)
    for j in range(len(features)):
        for index in range(0,len(feature[j])):
            # print(i, index)
            # print(feature[i][index])
            feature[j][index]=float(feature[j][index])

    return feature


In [45]:
# print('1st data',DataSet[2])
# print('2nd data',DataSet[0][1])
# print('3rd data',DataSet[0])
# print('4th data',DataSet[0][3])
# print('5th data',DataSet[0][4])
# print('6th data',DataSet[0][5])
# print('7th data',DataSet[0][6])
# for i in range(len(DataSet)):
#     DataSet[i][0]=DataSet[i][0][0]
# print(len(DataSet[0]))
x = features_append(DataSet)
# x = DataSet
# df = pd.DataFrame(x)
# for i in range(len(x)):
#     x[i] = x[i][0]
print('2nd',x[0])

[['0.3431890904903412', '0.07531614601612091', '1900.5250272933993', '2081.5942793422764', '3765.94189453125', '0.0849560546875', '-331.0139465332031', '107.94735717773438', '24.277204513549805', '11.558176040649414', '-19.487794876098633', '4.600085258483887', '-16.40422821044922', '-10.175797462463379', '-11.678380966186523', '-9.083390235900879', '-13.660134315490723', '-3.213951349258423', '-19.12630844116211', '-3.7342135906219482', '-0.3473721444606781', '-1.137442708015442', '-11.253403663635254', '0.32225072383880615', '1.6086572408676147', '-5.872960090637207']]
2nd [0.3431890904903412, 0.07531614601612091, 1900.5250272933993, 2081.5942793422764, 3765.94189453125, 0.0849560546875, -331.0139465332031, 107.94735717773438, 24.277204513549805, 11.558176040649414, -19.487794876098633, 4.600085258483887, -16.40422821044922, -10.175797462463379, -11.678380966186523, -9.083390235900879, -13.660134315490723, -3.213951349258423, -19.12630844116211, -3.7342135906219482, -0.34737214446067

In [46]:
X_train, X_test, y_train, y_test = train_test_split(x, all['Labels'], test_size=0.2, random_state=0)
# X_train = [X_train]
print(X_train[0][0])

0.3077155649662018


In [58]:
# clf = RandomForestClassifier(random_state=0, n_estimators=40)
clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)
clf.fit(X_train,y_train.values.ravel())

DecisionTreeClassifier(criterion='entropy', max_depth=3)

In [59]:
y_pred = clf.predict(X_test)

In [60]:
clf.predict(X_test)

array([1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0],
      dtype=int64)

In [61]:
y_test

85     1
438    0
96     1
172    1
134    1
      ..
379    0
185    1
76     1
395    0
313    0
Name: Labels, Length: 110, dtype: int64

In [62]:
a = accuracy_score(y_pred, y_test)
print(a)

0.6727272727272727


In [63]:
clf.predict(x[:12])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [64]:
all['Labels'][:12]

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
Name: Labels, dtype: int64

In [65]:
newTest = os.listdir("AllRecords/Test/")
Test = pd.DataFrame(newTest, columns={"Paths"})
l=os.path.join(os.path.abspath('AllRecords/Test/{0}')).format(Test['Paths'][9])
y,sr = librosa.load(l, res_type='kaiser_fast')

chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
rmse        = librosa.feature.rms(y=y)
spec_cent   = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw     = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff     = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr         = librosa.feature.zero_crossing_rate(y)
mfcc        = librosa.feature.mfcc(y=y, sr=sr)
to_append   = f'{np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'
for e in mfcc:
    to_append += f' {np.mean(e)}'
features=[]
features.append(to_append.split())
print(len(features[0]))

for counter in range(0,len(features[0])):
    features[0][counter]=float(features[0][counter])

get_feat = Test.apply(Extract_features, whichone='Test', axis=1)
combine = features_append(get_feat)
# print(combine[0])
print("Length", len(combine[9]))

26
[['0.47074562311172485', '0.006534410174936056', '1508.3056738658613', '1845.219788005738', '3159.6502488659276', '0.054908014112903226', '-533.5375366210938', '119.29576873779297', '-1.6009817123413086', '32.47652816772461', '-7.533818244934082', '17.795495986938477', '14.32162857055664', '1.421666145324707', '-15.793218612670898', '-1.4759856462478638', '-1.8549346923828125', '3.713296890258789', '5.0833964347839355', '0.881470263004303', '8.37099838256836', '16.475154876708984', '5.7274861335754395', '0.6608526110649109', '0.9324504137039185', '3.5562145709991455']]
Length 26


In [66]:
clf.predict(combine)

array([0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0], dtype=int64)

In [56]:
pickle.dump(clf, open("Sentence-model2.pkl", 'wb'))

In [57]:
clf = tree.DecisionTreeClassifier(random_state=42)
iris = load_iris()

clf = clf.fit(iris.data, iris.target)

dot_data = tree.export_graphviz(clf, out_file=None,
                                feature_names=iris.feature_names,
                                class_names=iris.target_names,
                                filled=True, rounded=True,
                                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data)

# empty all nodes, i.e.set color to white and number of samples to zero
for node in graph.get_node_list():
    if node.get_attributes().get('label') is None:
        continue
    if 'samples = ' in node.get_attributes()['label']:
        labels = node.get_attributes()['label'].split('<br/>')
        for i, label in enumerate(labels):
            if label.startswith('samples = '):
                labels[i] = 'samples = 0'
        node.set('label', '<br/>'.join(labels))
        node.set_fillcolor('white')

samples = iris.data[129:130]
decision_paths = clf.decision_path(samples)

for decision_path in decision_paths:
    for n, node_value in enumerate(decision_path.toarray()[0]):
        if node_value == 0:
            continue
        node = graph.get_node(str(n))[0]            
        node.set_fillcolor('green')
        labels = node.get_attributes()['label'].split('<br/>')
        for i, label in enumerate(labels):
            if label.startswith('samples = '):
                labels[i] = 'samples = {}'.format(int(label.split('=')[1]) + 1)

        node.set('label', '<br/>'.join(labels))

filename = 'tree.png'
graph.write_png(filename)

NameError: name 'tree' is not defined